In [1]:
import graphlab as gl
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt


This non-commercial license of GraphLab Create for academic use is assigned to surbhi.jain@sjsu.edu and will expire on November 21, 2018.


[INFO] graphlab.cython.cy_server: GraphLab Create v2.1 started. Logging: /tmp/graphlab_server_1511381048.log


In [9]:

def score(df_true, df_pred):

    df = pd.concat([df_pred,
                    df_true], axis=1)

    g = df.groupby('user_id')

    top_5 = g.pred_rating.apply(
        lambda x: x >= x.quantile(.95)
    )

    return df_true[top_5==1].mean()['true_rating']

def load_data():
    # Input data
    sf = gl.SFrame("../data/ratings.dat", format='tsv')

    # Data to test predictions on
    df_sample = pd.read_csv("../data/sample_submission.csv")
    sf_sample = gl.SFrame(df_sample)

    return sf, sf_sample, df_sample

def recommendation_modules(sf, num_factors, regularization = None):
    ranking_model = gl.recommender.ranking_factorization_recommender.create(observation_data=sf,
                                                     user_id="user_id",
                                                     item_id="joke_id",
                                                     target='rating',
                                                     solver='auto',
                                                     num_factors = num_factors,
                                                     regularization = regularization,
                                                     verbose = False,
                                                     random_seed = 42)
    
    factorization_model = gl.recommender.factorization_recommender.create(observation_data=sf,
                                                     user_id="user_id",
                                                     item_id="joke_id",
                                                     target='rating',
                                                     solver='auto',
                                                     num_factors = num_factors,
                                                     regularization = regularization,
                                                     verbose = False,
                                                     random_seed = 42)
    
    item_sim_model_pearson = gl.recommender.item_similarity_recommender.create(observation_data=sf,
                                                     user_id="user_id",
                                                     item_id="joke_id",
                                                     target='rating',
                                                     #solver='auto',
                                                     #num_factors = num_factors,
                                                     #regularization = regularization,
                                                     verbose = False,
                                                     #random_seed = 42, 
                                                     similarity_type='pearson')
    
    
    
    return ranking_model, factorization_model, item_sim_model_pearson


In [10]:
if __name__ == "__main__":
    sf, sf_sample, df_sample = load_data()

    training_data, validation_data = gl.recommender.util.random_split_by_user(sf, 'user_id', 'joke_id')

    df_true = pd.DataFrame()
    df_pred = pd.DataFrame()

    df_true['user_id'] = validation_data['user_id']
    df_true['joke_id'] = validation_data['joke_id']

    df_true['true_rating'] = validation_data['rating']

    # Plot scores vs num_factors
    num_factors = range(2,4)
    scores = []
    for n in num_factors:
        for m in recommendation_modules(training_data, num_factors = n):
            #m = create_factorization_recommender(training_data, num_factors = n)
            df_pred['pred_rating'] = m.predict(validation_data)
            rc = score(df_true, df_pred)
            scores.append(rc)
            print 'Num Factors:', n, ' Score:', rc
        print "\n\n\n"
    plt.plot(num_factors, scores)
    plt.xlabel('Number of Latent Features')
    plt.ylabel('Score')
    plt.title('Score vs Number of Latent Features')
    plt.show()


Finished parsing file /Users/ckhatri/Downloads/joke_project/data/ratings.dat

Parsing completed. Parsed 100 lines in 0.501376 secs.

------------------------------------------------------
Inferred types from first 100 line(s) of file as 
column_type_hints=[int,int,float]
If parsing fails due to incorrect types, you can correct
the inferred type list above and pass it to read_csv in
the column_type_hints argument
------------------------------------------------------


Finished parsing file /Users/ckhatri/Downloads/joke_project/data/ratings.dat

Parsing completed. Parsed 1218325 lines in 0.683904 secs.

...

Recsys training: model = ranking_factorization_recommender

Recsys training: model = factorization_recommender

Recsys training: model = item_similarity

Num Factors: 3  Score: 2.05624323593
Num Factors: 3  Score: 2.50023674242
Num Factors: 3  Score: 2.5413284632






ValueError: x and y must have same first dimension, but have shapes (2,) and (6,)